In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.utils import shuffle

TEST_SIZE = 10000

In [ ]:
X, y = fetch_openml("mnist_784", version=1, return_X_y = True, as_frame = False)
print(X.shape, y.shape)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = TEST_SIZE)
X_train, y_train = shuffle(X_train, y_train, random_state = 42)
X_test, y_test = shuffle(X_test, y_test, random_state = 42)
print(X_train.shape, y_train.shape)
d = 784
y_train = y_train.astype('int')
y_test = y_test.astype('int')
X_train = X_train / 255.0
X_test = X_test / 255.0

In [ ]:
# def init_params(): This is trash initialization
#     W1 = np.random.rand(d, 16) - 0.5
#     W2 = np.random.rand(16, 16) - 0.5
#     W3 = np.random.rand(16, 10) - 0.5
#     B1 = np.random.rand(1, 16) - 0.5
#     B2 = np.random.rand(1, 16) - 0.5
#     B3 = np.random.rand(1, 10) - 0.5
#     return W1, B1, W2, B2, W3, B3

def init_params():  # He initialization (take randn then * 2 / dim_of_one_data ;; 0 for all for biases)
    W1 = np.random.randn(784, 16) * np.sqrt(2.0 / 784) 
    B1 = np.zeros((1, 16))  # Bias init as zero  
    W2 = np.random.randn(16, 16) * np.sqrt(2.0 / 16)  
    B2 = np.zeros((1, 16))  # Bias init as zero  
    W3 = np.random.randn(16, 10) * np.sqrt(2.0 / 16)  
    B3 = np.zeros((1, 10))  # Bias init as zero  
    return W1, B1, W2, B2, W3, B3

def ReLU(Z):
    return np.maximum(0, Z)

def ReLU_derivative(Z):
    return np.where(Z > 0, 1, 0)

def stable_softmax(Z):
    e_Z = np.exp(Z - np.max(Z, axis = 1, keepdims = True)) # No keepdims = True causes np.max returning (N, ), keepdims -> (N, 1)
    return e_Z / np.sum(e_Z, axis = 1, keepdims = True)

def forward_propagation(W1, B1, W2, B2, W3, B3, X):
    A0 = X
    Z1 = A0.dot(W1) + B1
    A1 = ReLU(Z1)
    Z2 = A1.dot(W2) + B2
    A2 = ReLU(Z2)
    Z3 = A2.dot(W3) + B3
    A3 = stable_softmax(Z3)
    return A1, A2, A3, Z1, Z2, Z3

def one_hot(Y):
    # Y (batch_size, )
    # one_hot(Y): (batch_size, 10)
    one_hot = np.zeros((Y.size, 10))
    one_hot[np.arange(Y.size), Y] = 1
    return one_hot

def back_propagation(A1, A2, A3, Z1, Z2, Z3, W1, W2, W3, X, Y):
    tN = X.shape[0] # in case the last batch is insufficient
    dZ3 = A3 - Y
    dW3 = 1 / tN * A2.T.dot(dZ3)
    dB3 = np.mean(dZ3, axis = 0, keepdims = True)
    dA2 = dZ3.dot(W3.T)
    dZ2 = dA2 * ReLU_derivative(Z2) # Hadamard product (N x 16) x (N x 16) = (N x 16)
    dW2 = 1 / tN * A1.T.dot(dZ2)
    dB2 = np.mean(dZ2, axis = 0, keepdims = True)
    dA1 = dZ2.dot(W2.T)
    dZ1 = dA1 * ReLU_derivative(Z1)
    dW1 = 1 / tN * X.T.dot(dZ1)
    dB1 = np.mean(dZ1, axis = 0, keepdims = True)
    return dB1, dB2, dB3, dW1, dW2, dW3

def predict(W1, B1, W2, B2, W3, B3, X):
    # X (M x 784)
    A0 = X
    Z1 = A0.dot(W1) + B1
    A1 = ReLU(Z1)
    Z2 = A1.dot(W2) + B2
    A2 = ReLU(Z2)
    Z3 = A2.dot(W3) + B3
    A3 = stable_softmax(Z3)
    return A3

def loss(Y_pred, Y):
    # Y (all training imgs, )
    # Y_pred (all training imgs, 10)
    id0 = np.arange(Y.size)
    return -np.mean(np.log(Y_pred[id0, Y])) # np.log output ra 1 vector va lay mean cua vector do va khong keepdims thi ra np 1D?

def accuracy(Y_pred, Y):
    # Y (all training imgs, )
    # Y_pred (all training imgs, 10)
    Y_pred = np.argmax(Y_pred, axis = 1)
    cnt = np.sum(Y_pred == Y)
    return cnt * 100 / Y.size

def mini_batch_GD(W1, B1, W2, B2, W3, B3, lr = 0.01, batch_size = 10, num_epoch = 100):
    loss_hist = []
    acc_hist = []
    num_batch = int(X_train.shape[0] / batch_size)
    for epoch in range(num_epoch):
        mix_id = np.random.permutation(X_train.shape[0])
        for i in range(num_batch):
            batch_ids = mix_id[batch_size * i : min(X_train.shape[0], batch_size * (i + 1))]
            X_batch = X_train[batch_ids]
            y_batch = y_train[batch_ids]
            one_hot_Y = one_hot(y_batch)
            A1, A2, A3, Z1, Z2, Z3 = forward_propagation(W1, B1, W2, B2, W3, B3, X_batch)
            dB1, dB2, dB3, dW1, dW2, dW3 = back_propagation(A1, A2, A3, Z1, Z2, Z3, W1, W2, W3, X_batch, one_hot_Y)
            W1 = W1 - lr * dW1
            W2 = W2 - lr * dW2
            W3 = W3 - lr * dW3
            B1 = B1 - lr * dB1
            B2 = B2 - lr * dB2
            B3 = B3 - lr * dB3
        Y_pred = predict(W1, B1, W2, B2, W3, B3, X_train)
        loss_hist.append(loss(Y_pred, y_train))
        acc_hist.append(accuracy(Y_pred, y_train))
        print(f"Epoch: {epoch}; accuracy = {acc_hist[-1]:.2f}%")
    return W1, B1, W2, B2, W3, B3, loss_hist, acc_hist

W1, B1, W2, B2, W3, B3 = init_params()

In [ ]:
W1, B1, W2, B2, W3, B3, loss_hist, acc_hist = mini_batch_GD(W1, B1, W2, B2, W3, B3)

In [ ]:
Y_pred = predict(W1, B1, W2, B2, W3, B3, X_test)
print(f"Accuracy on testing data: {accuracy(Y_pred, y_test):.2f}%")

In [ ]:
Y_hat = np.argmax(Y_pred, axis = 1)
pred_errors = np.where(Y_hat != y_test)
pred_errors = np.array(pred_errors).reshape(-1)
pic_errors = X_test[pred_errors]
label_errors = y_test[pred_errors]
label_pred = Y_hat[pred_errors]
fig, ax = plt.subplots(ncols = 5, nrows = 5, figsize = (15, 15))
c = 0
for i in range(0, 5):
    for j in range(0, 5):
        ax[i, j].imshow(pic_errors[c].reshape((28, 28)))
        ax[i, j].set(title = f"True label: {label_errors[c]}; pred: {label_pred[c]}")
        ax[i, j].axis('off')
        c += 1
        

Kiến thức:
- np.arange(N): tạo mảng np [0, 1,..N]  (có thể thành np.arange(N, d)) với d là bước nhảy
- range(N): tương tự như trên nhưng không có bước nhảy và tạo ra range object (not a list)
- np.where(Z > 0): trả về index của vị trí có ptu > 0, nếu Z 1D thì ok, 2D thì trả về kiểu ma trận thưa. Nhưng nếu np.where(Z > 0, 1, 0) thì lại trả về nguyên thằng Z, với ptu tại vị trí i, j = 1 nếu Z[i, j] > 0 và ngược lại. Tóm gọn là chỉ ghi đơn giản như np.where(Z > 0) thì y như đang hỏi Z là index nào thỏa mãn? thì trả về những index đó, còn np.where(Z > 0, 1, 0) thì là bảo Z chỗ nào > 0 thì thành 1 và bé hơn 0 thì thành 0 nhé. np.any(Z > 0, axis = 1) != np.any(Z > 0) or np.all(Z > 0).
- (dim 1, dim 2, dim 3,...) -> axis 0, axis 1,...
- He initialization: Đối với những mạng có ReLU, Bias nên khởi tạo = 0, còn weight thì randn xong nhân với 2 / dims_of_a_data_point